In [13]:
from see.Segment_Fitness import FitnessFunction, multi_value_ff
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness
from see import base_classes, GeneticSearch
import copy
import numpy as np
#FitnessFunction(datas[3].mask,datas[1].mask)

In [14]:
def GeneratePairs(size): #Generates each pair in a set
    pairs=[]
    for i in range(size):
        for j in range(i+1,size):
            pairs.append([i,j])
    return pairs
GeneratePairs(4)

[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]

In [15]:

def UncertaintyValue(segmenters,data): #Compute uncertainty value
    #Generate N copies of image
    dataCopies=[copy.deepcopy(data) for i in range(len(segmenters))]

    #Create segmenters
    segs=[workflow(i) for i in segmenters]

    #Run each segmenter on a copy of an image
    dataCopies=[segs[i].runAlgo(dataCopies[i]) for i in range(len(segs))]

    #Return the mean difference between each pairwise prediction
    uncertainty=mean([FitnessFunction(dataCopies[i[0]],dataCopies[i[1]]) for i in GeneratePairs(len(dataCopies))])
    return uncertainty

UncertaintyValue

<function __main__.UncertaintyValue(segmenters, data)>

In [16]:
def ALSearch(segmenters,dataSet): #Search data set for image with highest uncertainty value
    #Generate uncertainty value for each image
    uncertainties=[UncertaintyValue(segmenters,data) for data in dataSet]

    #Return copy of image with most uncertainty (should this be argmax?)
    return np.argmin(uncertainties) #copy.deepcopy(dataSet[np.argmin(uncertainties)])

In [17]:
def ActiveLearning(trainingDataSet,testData,iterations=10):
    #Initialize selected training set and fitness tracking
    selectedTrainSet=[]
    testFitness=[]
    best=[]

    #Copy training set to unselected set
    unselectedTrainingData=copy.deepcopy(trainingDataSet)

    #Select a single image from training set to initialize active training set and remove from unselected set
    selectedInt=np.random.randint(0,len(unselectedTrainingData))
    selectedTrainSet.append(unselectedTrainingData[selectedInt])
    del unselectedTrainingData[selectedInt]

    #Define workflow and initialize evolver
    workflow.addalgos([colorspace, segmentor, segment_fitness])
    my_evolver = GeneticSearch.Evolver(workflow, selectedTrainSet, pop_size=100)

    #Begin loop over active learning iterations
    for i in range(iterations): 
        #Execute first round of evolution ??How do you modify the training data??
        population = my_evolver.run(ngen=10)

        #Generate model ensemble from quality models in the population (HoF)
        ensemble=[]
        ensemble=my_evolver.hof[:min(10,len(my_evolver.hof))]

        #Use ALSearch to maximize uncertainty over unselected training set
        selectedImage=ALSearch(ensemble,unselectedTrainingData)
        

        #Add new sample to training set and remove from unselected
        selectedTrainSet.append(unselectedTrainingData[selectedImage])
        del unselectedTrainingData[selectedImage]

        #Update evolver with new data
        my_evolver.updateData(selectedTrainSet)

        #Record current best performance on test data and append to tracking list
        tData=copy.deepcopy(testData)
        seg=workflow(my_evolver.hof[0])
        tData=seg.runAlgo(tData)
        bestFitness=multi_value_ff(tData)
        testFitness.append(bestFitness)
        

    #Return best model found, best fitness found, and fitness list
    return best, bestFitness, testFitness

In [4]:
#The following code will download the dataset to the Images_Data folder
from see import DataDownload as dd

dd.downloadKOMATSUNA(folder='./Image_Data/KOMATSUNA/')

Directory  ./Image_Data/KOMATSUNA/  Created 
Unzipping rgbd_plant.zip
Download and Convert of rgbd_plant.zip Complete
Unzipping rgbd_label.zip
Download and Convert of rgbd_label.zip Complete


In [5]:
from imageio import v3 as imageio

img1 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_00.png')
gmask1 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_00.png')

img2 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_01.png')
gmask2 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_01.png')

img3 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_02.png')
gmask3 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_02.png')

img4 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_03.png')
gmask4 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_03.png')

In [6]:
from see import base_classes

# put data in a pipeline
mydata = base_classes.pipedata()
mydata.append(img1)
mydata.append(img2)
mydata.append(img3)
mydata.append(img4)
mydata.gtruth = []
mydata.gtruth.append(gmask1)
mydata.gtruth.append(gmask2)
mydata.gtruth.append(gmask3)
mydata.gtruth.append(gmask4)

In [18]:
output=ActiveLearning(mydata, mydata, iterations=1)

Initializing a new random population
Generation 0/10 of population size 100
<class 'see.Workflow.workflow'> parameters: 
	colorspace = YIQ
	multichannel = False
	channel = 1
	algorithm = Chan_Vese
	alpha1 = 0.375
	alpha2 = 0.6875
	beta1 = 0.46875
	beta2 = 0.47265625
	gamma1 = 0.703125
	gamma2 = 0.14453125
	n_segments = 4
	max_num_iter = 19



AttributeError: 'numpy.ndarray' object has no attribute 'append'